In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt  


## This is a demo file for pyCCUS test drive, designed for JD

#### Step 1: have your CMG model ready as the input to pyCCUS
#### Step 2: get your parameter space ready
- This example demo alters the values of permeability, porosity, and injection rate

In [2]:
##### User input #####
# folder_path = '../data/omv/CCS/testdrive1'
folder_path = '..\\data\\omv\\CCS\\testdrive1'

params = ['phi',
          'perm_mult, md',
          'inj_rate, m3/d']

l_bounds = [0.15, 8000, 1e6]
u_bounds = [0.25, 11000, 9e6]

num_of_exp = 3
##### User input #####

In [3]:
##### Latin Hyper Cube sampling to create the params space #####
from scipy.stats import qmc
sampler = qmc.LatinHypercube(d=len(params))
sample = sampler.random(n=num_of_exp)

sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

df = pd.DataFrame(data=np.round(sample_scaled,2), columns=params)

df['perm, md'] = df['phi'] * df['perm_mult, md']

##### Save your exp design if needed #####
if not os.path.isdir(folder_path):
    os.makedirs(folder_path)
df.to_csv(os.path.join(folder_path, 'exp_design.csv'), index=False)

ImportError: cannot import name 'qmc' from 'scipy.stats' (C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\__init__.py)

- Write CMG models (dat files) based on the DoE

In [3]:
df = pd.read_csv(os.path.join(folder_path, 'exp_design.csv'))
df_input = df.iloc[:,[0,2,3]]
df_input = df_input.rename(columns={'phi': 'phi', 
                                    'inj_rate, m3/d': 'inj_rate',
                                    'perm, md': 'perm'})

In [4]:
import sys
# append the path of the parent directory
sys.path.append("..")
# import method from sibling module
from utils.pyCMG_Model import omv_CCS
from utils.pySherlock import pysherlock

In [5]:
omvccs = omv_CCS()
omvccs.folder_path = folder_path
omvccs.title1 = 'pyCCUS testdrive'
omvccs.title2 = 'CCS omv'
omvccs.title3 = 'JD+YL'

In [7]:
omvccs.write_simfiles(df_input=df_input, verbose=True)

Job done -- write 3 CMG dat files based on exp design csv .....


## Run CMG files
- Option 1: local win machine
- Option 2: Stanford sherlock HPC

In [9]:
from utils.pyCMG_Control import pycmgcontrol

for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10'
    pycmg_ctrl.cmg_version = 'ese-ts1win-v2023.20'

    pycmg_ctrl.run_gem_simulation(case_name_suffix=f'case{nn+1}.dat')

### Option 2: run CMG on Sherlock
- You will also need submit.sh file

In [ ]:
# sherlock = pysherlock()
# pyCTRL_folder_path = os.path.join(folder_path, 'pyCTRLfiles')

# for idx in range(df_input.shape[0]):
#     sherlock.write_pyCTRLfile(folder_path=pyCTRL_folder_path, caseid=idx+1)

### Extract the result in npy format

In [17]:
for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10'
    pycmg_ctrl.cmg_version = 'ese-ts1win-v2023.20'
    pycmg_ctrl.rwd_precis = 4
    pycmg_ctrl.proplist = ['SG','PRES']
    pycmg_ctrl.layer_nums = [1]
    pycmg_ctrl.time_query = [2024, 2026, 2028, 2030, 2032, 2034]

    pycmg_ctrl.cmgrst2npy(caseid=f"{nn+1}",
                              verbose=False, 
                              rwodelete=True)
    npy_data = pycmg_ctrl.cmg2npy

#     # try datfiles or simfiles depending on different computing env
#     if not os.path.isdir(os.path.join(folder_path, 'datfiles', 'rst_npy')):
#         os.makedirs(os.path.join(folder_path, 'datfiles', 'rst_npy'))
#     np.save(os.path.join(folder_path, 'datfiles', 'rst_npy', f'case{nn+1}.npy'), arr = npy_data)